### **Studi Kasus**
Kumpulan data yang akan gunakan untuk studi kasus ini dikenal sebagai kumpulan data Kanker Payudara Wisconsin. 
Tugas yang terkait dengannya adalah Klasifikasi. Dataset berisi total 10 fitur yang diberi label dalam kelas jinak atau ganas. 
Fitur memiliki 699 contoh di mana 16 nilai fitur tidak ada. Dataset hanya berisi nilai numerik.
Kita akan mengimplementasikan Ensemble menggunakan perpustakaan scikit-learn yang hebat.


Langkah Pertama adalah menginstall dependency dan import library

In [ ]:
pip install --exists-action=i scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

Kemudian kita Load dataset, dan coba apakah dataset sudah bisa diakses atau belum

In [ ]:
data = pd.read_excel('https://github.com/SyahrulRamadhani21/Ensemble-Learning/blob/main/cancer.xlsx?raw=true', sheet_name='breast-cancer-wisconsin(1)')
data.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


Selanjutnya mari kita melihat statistik dari data dengan fungsi describe() dan info() 

In [ ]:
data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Clump Thickness              699 non-null    int64 
 1   Uniformity of Cell Size      699 non-null    int64 
 2   Uniformity of Cell Shape     699 non-null    int64 
 3   Marginal Adhesion            699 non-null    int64 
 4   Single Epithelial Cell Size  699 non-null    int64 
 5   Bare Nuclei                  699 non-null    object
 6   Bland Chromatin              699 non-null    int64 
 7   Normal Nucleoli              699 non-null    int64 
 8   Mitoses                      699 non-null    int64 
 9   Class                        699 non-null    int64 
dtypes: int64(9), object(1)
memory usage: 54.7+ KB


Seperti disebutkan sebelumnya, dataset berisi nilai yang hilang. Kolom bernama "Bare Nuclei" berisi mereka. Mari kita verifikasi.

In [ ]:
data['Bare Nuclei']

0       1
1      10
2       2
3       4
4       1
       ..
694     2
695     1
696     3
697     4
698     5
Name: Bare Nuclei, Length: 699, dtype: object

Kita dapat melihat beberapa "?" ada di dalamnya, kan? Nah, ini adalah nilai-nilai data yang hilang, dan Kita akan menghubungkannya dengan Mean Imputation. Tapi pertama-tama, Kita akan mengganti "?" itu dengan 0.

In [ ]:
data.replace('?',0, inplace=True)
data['Bare Nuclei']

0       1
1      10
2       2
3       4
4       1
       ..
694     2
695     1
696     3
697     4
698     5
Name: Bare Nuclei, Length: 699, dtype: object

"?" diganti dengan 0 sekarang. Mari kita lakukan pengobatan nilai yang hilang sekarang.

In [ ]:
# Convert the DataFrame object into NumPy array otherwise you will not be able to impute
values = data.values

# Now impute it
imputer = SimpleImputer()
imputedData = imputer.fit_transform(values)

Sekarang jika Kita melihat dataset itu sendiri, Kita akan melihat bahwa semua rentang fitur dari dataset tidak sama. Ini dapat menyebabkan masalah. Perubahan kecil pada fitur mungkin tidak memengaruhi fitur lainnya. Untuk mengatasi masalah ini, Kita akan menormalkan rentang fitur ke rentang yang seragam, dalam hal ini, 0 - 1.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
normalizedData = scaler.fit_transform(imputedData)

Kita sudah selesai melakukan pemprosesan untuk eksperimen Ensembling Kita.



---

Kita akan mulai dengan Ensembling berbasis Bagging. Dalam hal ini, Kita akan menggunakan Pohon Keputusan yang Dikantongi.

In [ ]:
# Bagged Decision Trees for Classification - necessary dependencies

from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

Kita telah mengimpor dependensi untuk Bagged Decision Trees.

In [ ]:
# Segregate the features from the labels
X = normalizedData[:,0:9]
Y = normalizedData[:,9]

In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=7)
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.9584886128364388


Pertama, Kita menginisialisasi lipatan validasi silang 10 kali lipat. Setelah itu, Kita membuat instance Decision Tree Classifier dengan 100 pohon dan membungkusnya dalam Ensemble berbasis Bagging. Kemudian Kita mengevaluasi model Kita.

Model kita tampil cukup baik. Ini menghasilkan akurasi 95,84%.

In [ ]:
# AdaBoost Classification

from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 70
kfold = model_selection.KFold(n_splits=10, random_state=seed,shuffle=True)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.9599378881987578


Dalam hal ini, Anda melakukan klasifikasi AdaBoost (dengan 70 pohon) yang didasarkan pada jenis Ensembling Boosting. Model ini memberi Anda akurasi 95,99% untuk validasi silang 10 kali lipat.



---
Terakhir, saatnya Anda menerapkan teknik Ensemble berbasis Voting.


In [ ]:
# Voting Ensemble for Classification

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

kfold = model_selection.KFold(n_splits=10, random_state=seed,shuffle=True)
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, Y, cv=kfold)
print(results.mean())

0.9642028985507245


Kita menerapkan model Ensemble berbasis Voting di mana Kita menggunakan Regresi Logistik, Pohon Keputusan, dan Mesin Vektor Dukungan untuk tujuan pemungutan suara. Model melakukan yang terbaik sejauh ini dengan akurasi 96,42% untuk validasi silang 10 kali lipat.